Install the required libraries

In [ ]:
!pip install tenseal psycopg2

## Generating Keys

Before that Have to create a table inside the database named NewDatabase

```sql 
CREATE TABLE encryption_keys (
     id SERIAL PRIMARY KEY,
     secret_key BYTEA,
     public_key BYTEA
 );
```

In below code change the following fields as per your database

`dbname`
`user`
`password`
`host`
`port`

In [6]:
import tenseal as ts
import psycopg2

# Step 1: Initialize Homomorphic Encryption Context
context = ts.context(
    ts.SCHEME_TYPE.BFV,
    poly_modulus_degree=8192,
    plain_modulus=1032193,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)

context.generate_galois_keys()
context.generate_relin_keys()
context.global_scale = 2**40
context.auto_rescale = True

# Generate Keys
secret_key = context.secret_key()
public_context = context.copy()
public_context.make_context_public()

# Serialize secret key (private) and public context (without secret key)
secret_key_bytes = context.serialize(save_secret_key=True)
public_key_bytes = public_context.serialize()

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

# Store as BYTEA
cur.execute(
    "INSERT INTO encryption_keys (secret_key, public_key) VALUES (%s, %s)",
    (psycopg2.Binary(secret_key_bytes), psycopg2.Binary(public_key_bytes))
)

conn.commit()
cur.close()
conn.close()


## Inserting Data

Before inserting the inside the database create the following table inside each database

```sql
 CREATE TABLE encrypted_crime_reports (
     id SERIAL PRIMARY KEY,  -- Auto-increment ID
     encryption_key_id INTEGER,
     crime_type BYTEA,  -- Encrypted crime type (as BYTEA)
     location BYTEA,  -- Encrypted location (as BYTEA)
     description BYTEA,  -- Encrypted description (as BYTEA)
     date BYTEA,  -- Encrypted date (as BYTEA)
     time BYTEA,  -- Encrypted time (as BYTEA)
     criminal_name BYTEA,  -- Encrypted criminal name (as BYTEA)
     encrypted_crime_type BYTEA  -- Encrypted vectorized crime type (as BYTEA)
 );
```
and run the code for inserting the db for each database by changing the following fields

`dbname`
`user`
`password`
`host`
`port`.

### DB 4

In [ ]:
# 📝 Step 4: Crime Report to Encrypt
crime_reports = [
  { "crime_type": "Identity Theft", "location": "Portland", "description": "Bank account compromised through phishing email", "date": "2024-02-23", "time": "14:35", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Chicago", "description": "Entry through unlocked basement window", "date": "2024-03-18", "time": "03:42", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Atlanta", "description": "Luxury vehicle stolen using cloned key fob", "date": "2024-01-09", "time": "19:15", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Boston", "description": "Victim attacked while walking dog in park", "date": "2024-04-03", "time": "06:20", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Fraud", "location": "Dallas", "description": "Fake investment scheme targeting retirees", "date": "2024-03-27", "time": "Unknown", "criminal_name": "Victoria Stephens" },
  { "crime_type": "Shoplifting", "location": "Seattle", "description": "Designer clothing concealed in modified bag", "date": "2024-02-11", "time": "13:50", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Denver", "description": "Multiple vehicles keyed in apartment complex", "date": "2024-01-28", "time": "02:10", "criminal_name": "Unknown" },
  { "crime_type": "Drug Possession", "location": "Miami", "description": "Substances found in airport security check", "date": "2024-04-05", "time": "09:25", "criminal_name": "Unknown" },
  { "crime_type": "DUI", "location": "Phoenix", "description": "Driver crashed into parked vehicles", "date": "2024-03-15", "time": "01:30", "criminal_name": "Timothy Wright" },
  { "crime_type": "Cyber Attack", "location": "Minneapolis", "description": "Healthcare provider database compromised", "date": "2024-02-09", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Philadelphia", "description": "Jewelry store robbed during business hours", "date": "2024-01-17", "time": "16:40", "criminal_name": "Jacob Torres" },
  { "crime_type": "Trespassing", "location": "San Francisco", "description": "Individual found in secure area of government building", "date": "2024-03-22", "time": "11:05", "criminal_name": "Unknown" },
  { "crime_type": "Embezzlement", "location": "Houston", "description": "Accountant diverted funds to personal account", "date": "2024-04-01", "time": "Unknown", "criminal_name": "Rebecca Collins" },
  { "crime_type": "Identity Theft", "location": "Detroit", "description": "Mail stolen from multiple residences", "date": "2024-02-15", "time": "06:50", "criminal_name": "Unknown" },
  { "crime_type": "Stalking", "location": "New Orleans", "description": "Victim harassed through social media accounts", "date": "2024-01-23", "time": "22:15", "criminal_name": "Unknown" },
  { "crime_type": "Arson", "location": "Indianapolis", "description": "Vehicle set on fire in driveway", "date": "2024-03-29", "time": "03:45", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Weapon Possession", "location": "Las Vegas", "description": "Concealed weapon found during security screening", "date": "2024-02-08", "time": "20:30", "criminal_name": "Unknown" },
  { "crime_type": "Blackmail", "location": "Nashville", "description": "Executive threatened over compromising photos", "date": "2024-01-31", "time": "Unknown", "criminal_name": "Andrea Parker" },
  { "crime_type": "Counterfeiting", "location": "San Diego", "description": "Counterfeit bills used at multiple locations", "date": "2024-03-12", "time": "15:20", "criminal_name": "Unknown" },
  { "crime_type": "Child Endangerment", "location": "Austin", "description": "Children found unsupervised in hotel room", "date": "2024-02-27", "time": "23:10", "criminal_name": "Unknown" },
  { "crime_type": "Extortion", "location": "Cincinnati", "description": "Small business threatened for protection money", "date": "2024-01-19", "time": "17:45", "criminal_name": "Marcus Henderson" },
  { "crime_type": "Hit and Run", "location": "Pittsburgh", "description": "Pedestrian injured in crosswalk", "date": "2024-03-14", "time": "18:55", "criminal_name": "Unknown" },
  { "crime_type": "Public Intoxication", "location": "Buffalo", "description": "Disruptive behavior at public event", "date": "2024-02-18", "time": "21:40", "criminal_name": "Unknown" },
  { "crime_type": "Forgery", "location": "Cleveland", "description": "Forged prescription at pharmacy", "date": "2024-01-26", "time": "12:15", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Dumping", "location": "Louisville", "description": "Construction debris left in public park", "date": "2024-03-05", "time": "05:30", "criminal_name": "Unknown" },
  { "crime_type": "Human Trafficking", "location": "Tampa", "description": "Workers held in deplorable conditions", "date": "2024-02-02", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Money Laundering", "location": "Tucson", "description": "Front business used to clean drug money", "date": "2024-01-11", "time": "Unknown", "criminal_name": "Nathaniel Rogers" },
  { "crime_type": "Kidnapping", "location": "Columbus", "description": "Child briefly taken by non-custodial parent", "date": "2024-03-25", "time": "16:20", "criminal_name": "Melissa Turner" },
  { "crime_type": "Harassment", "location": "Kansas City", "description": "Continuous unwanted contact after restraining order", "date": "2024-02-14", "time": "10:05", "criminal_name": "Unknown" },
  { "crime_type": "Tax Evasion", "location": "Memphis", "description": "Business owner hiding income", "date": "2024-01-29", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Gambling", "location": "Raleigh", "description": "Betting operation run out of bar basement", "date": "2024-03-09", "time": "22:50", "criminal_name": "Samuel Brooks" },
  { "crime_type": "Credit Card Fraud", "location": "Milwaukee", "description": "Skimming device installed on ATM", "date": "2024-02-17", "time": "04:15", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Sacramento", "description": "Road rage incident escalated to violence", "date": "2024-01-15", "time": "17:35", "criminal_name": "Unknown" },
  { "crime_type": "Drug Distribution", "location": "St. Louis", "description": "Dealer arrested during undercover operation", "date": "2024-03-31", "time": "23:25", "criminal_name": "Derrick Powell" },
  { "crime_type": "Identity Theft", "location": "Jacksonville", "description": "Victim's information used to file tax return", "date": "2024-02-22", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Omaha", "description": "Network operating out of massage parlor", "date": "2024-01-13", "time": "15:50", "criminal_name": "Unknown" },
  { "crime_type": "Bribery", "location": "Honolulu", "description": "Inspector accepted payment to overlook violations", "date": "2024-03-17", "time": "14:40", "criminal_name": "Harold Mitchell" },
  { "crime_type": "Burglary", "location": "Charlotte", "description": "Office equipment stolen after hours", "date": "2024-02-20", "time": "01:55", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Providence", "description": "Rental car not returned after agreement period", "date": "2024-01-21", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Insider Trading", "location": "Richmond", "description": "Employee traded on merger information", "date": "2024-03-07", "time": "09:35", "criminal_name": "Lawrence Grant" },
  { "crime_type": "Child Abuse", "location": "Birmingham", "description": "Injuries discovered during medical exam", "date": "2024-02-05", "time": "11:20", "criminal_name": "Unknown" },
  { "crime_type": "Grand Theft", "location": "Oklahoma City", "description": "Artwork stolen from gallery opening", "date": "2024-01-25", "time": "20:45", "criminal_name": "Unknown" },
  { "crime_type": "Domestic Violence", "location": "Albuquerque", "description": "Victim hospitalized after altercation", "date": "2024-03-21", "time": "23:50", "criminal_name": "Unknown" },
  { "crime_type": "Impersonation", "location": "Fort Worth", "description": "Scammer posed as utility worker", "date": "2024-02-25", "time": "13:40", "criminal_name": "Unknown" },
  { "crime_type": "Cyberbullying", "location": "Rochester", "description": "Teen targeted through school social platforms", "date": "2024-01-07", "time": "19:25", "criminal_name": "Unknown" },
  { "crime_type": "Insurance Fraud", "location": "Madison", "description": "Staged accident for policy payout", "date": "2024-03-11", "time": "08:10", "criminal_name": "Gabrielle Foster" },
  { "crime_type": "Shoplifting", "location": "Baton Rouge", "description": "Organized group targeting electronics store", "date": "2024-02-13", "time": "16:30", "criminal_name": "Unknown" },
  { "crime_type": "Reckless Driving", "location": "Norfolk", "description": "Excessive speed in construction zone", "date": "2024-01-30", "time": "07:55", "criminal_name": "Unknown" },
  { "crime_type": "Animal Cruelty", "location": "Des Moines", "description": "Neglected animals discovered in home", "date": "2024-03-28", "time": "10:25", "criminal_name": "Unknown" },
  { "crime_type": "Carjacking", "location": "New York", "description": "Driver forced out at gunpoint", "date": "2024-02-29", "time": "22:05", "criminal_name": "Unknown" },
  { "crime_type": "Online Scam", "location": "Tulsa", "description": "Fake charity collecting disaster relief", "date": "2024-01-05", "time": "Unknown", "criminal_name": "Jessica Coleman" },
  { "crime_type": "Vandalism", "location": "Fresno", "description": "Historic monument defaced with paint", "date": "2024-03-23", "time": "00:20", "criminal_name": "Unknown" },
  { "crime_type": "Ponzi Scheme", "location": "Salt Lake City", "description": "Investment fraud targeting church members", "date": "2024-02-07", "time": "Unknown", "criminal_name": "Douglas Price" },
  { "crime_type": "Theft of Services", "location": "Spokane", "description": "Cable service illegally connected", "date": "2024-01-03", "time": "14:15", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Anchorage", "description": "False documentation for government benefits", "date": "2024-03-19", "time": "09:50", "criminal_name": "Unknown" },
  { "crime_type": "Unlawful Possession of Firearm", "location": "Wichita", "description": "Weapon found during traffic violation stop", "date": "2024-02-16", "time": "21:35", "criminal_name": "Aaron Carter" },
  { "crime_type": "Identity Theft", "location": "Orlando", "description": "Hotel rooms booked with stolen information", "date": "2024-01-14", "time": "18:30", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Oakland", "description": "Food delivery driver robbed of cash", "date": "2024-03-16", "time": "21:15", "criminal_name": "Unknown" },
  { "crime_type": "Loitering", "location": "Jersey City", "description": "Group refused to leave business after closing", "date": "2024-02-03", "time": "23:55", "criminal_name": "Multiple Unknown" },
  { "crime_type": "DUI", "location": "Fort Lauderdale", "description": "Driver found asleep at stoplight", "date": "2024-01-22", "time": "02:40", "criminal_name": "Christina Morris" },
  { "crime_type": "Check Fraud", "location": "Montgomery", "description": "Altered check amounts after receiving", "date": "2024-03-26", "time": "15:05", "criminal_name": "Unknown" },
  { "crime_type": "Child Neglect", "location": "Lincoln", "description": "Young children left home alone overnight", "date": "2024-02-26", "time": "06:40", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Lexington", "description": "Operation discovered during hotel investigation", "date": "2024-01-10", "time": "22:45", "criminal_name": "Unknown" },
  { "crime_type": "Corporate Espionage", "location": "Durham", "description": "Research stolen from biotech company", "date": "2024-03-04", "time": "Unknown", "criminal_name": "Wesley Hammond" },
  { "crime_type": "Assault", "location": "Boise", "description": "Concert venue altercation requiring hospitalization", "date": "2024-02-12", "time": "23:15", "criminal_name": "Unknown" },
  { "crime_type": "Perjury", "location": "Winston-Salem", "description": "False testimony in insurance case", "date": "2024-01-18", "time": "10:30", "criminal_name": "Unknown" },
  { "crime_type": "Drug Trafficking", "location": "Greensboro", "description": "Package intercepted containing narcotics", "date": "2024-03-10", "time": "12:50", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Laredo", "description": "Vehicle taken from dealership test drive", "date": "2024-02-21", "time": "16:15", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Lubbock", "description": "Home invaded while family on vacation", "date": "2024-01-08", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Harassment", "location": "Irving", "description": "Unwanted contact despite cease and desist", "date": "2024-03-30", "time": "19:40", "criminal_name": "Eric Sullivan" },
  { "crime_type": "Identity Theft", "location": "Scottsdale", "description": "Medical services billed under victim's name", "date": "2024-02-19", "time": "11:25", "criminal_name": "Unknown" },
  { "crime_type": "Trespassing", "location": "St. Petersburg", "description": "Individual found sleeping in private building", "date": "2024-01-27", "time": "05:10", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Anaheim", "description": "Business windows broken overnight", "date": "2024-03-08", "time": "01:05", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Irvine", "description": "Contractor took payment without performing work", "date": "2024-02-04", "time": "Unknown", "criminal_name": "Patrick Walters" },
  { "crime_type": "Shoplifting", "location": "Glendale", "description": "Multiple items concealed in stroller", "date": "2024-01-16", "time": "14:20", "criminal_name": "Unknown" },
  { "crime_type": "Cyber Attack", "location": "Huntsville", "description": "Ransomware affecting city systems", "date": "2024-03-24", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Drug Manufacturing", "location": "Chesapeake", "description": "Lab discovered in storage unit", "date": "2024-02-01", "time": "16:55", "criminal_name": "Unknown" },
  { "crime_type": "Criminal Mischief", "location": "Arlington", "description": "Mailboxes destroyed along residential street", "date": "2024-01-04", "time": "03:25", "criminal_name": "Unknown" },
  { "crime_type": "Disorderly Conduct", "location": "Gilbert", "description": "Disruption at school board meeting", "date": "2024-03-13", "time": "19:10", "criminal_name": "Martin Reynolds" },
  { "crime_type": "Larceny", "location": "Chandler", "description": "Wallet taken from gym locker", "date": "2024-02-24", "time": "17:25", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Immigration", "location": "Chula Vista", "description": "Falsified work documents discovered", "date": "2024-01-20", "time": "08:45", "criminal_name": "Unknown" },
  { "crime_type": "Homicide", "location": "Riverside", "description": "Victim found in vehicle with gunshot wound", "date": "2024-03-01", "time": "00:35", "criminal_name": "Unknown" },
  { "crime_type": "Identity Theft", "location": "Santa Ana", "description": "Multiple accounts opened using stolen identity", "date": "2024-02-28", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Breaking and Entering", "location": "San Bernardino", "description": "Forced entry through kitchen window", "date": "2024-01-06", "time": "20:15", "criminal_name": "Unknown" },
  { "crime_type": "Copyright Infringement", "location": "Stockton", "description": "Counterfeit merchandise sold at flea market", "date": "2024-03-03", "time": "13:05", "criminal_name": "Unknown" },
  { "crime_type": "Fare Evasion", "location": "Hialeah", "description": "Individual jumped transit barrier", "date": "2024-02-10", "time": "07:35", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Fremont", "description": "Patient attacked healthcare worker", "date": "2024-01-12", "time": "15:30", "criminal_name": "Franklin Jenkins" },
  { "crime_type": "Credit Card Theft", "location": "Garland", "description": "Card stolen from workplace and used", "date": "2024-03-20", "time": "12:25", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Firearms Sale", "location": "Arlington", "description": "Unregistered weapons sold at gun show", "date": "2024-02-06", "time": "11:40", "criminal_name": "Unknown" },
  { "crime_type": "Pickpocketing", "location": "Irving", "description": "Wallet stolen on crowded public transit", "date": "2024-01-02", "time": "08:20", "criminal_name": "Unknown" },
  { "crime_type": "Child Pornography", "location": "Corpus Christi", "description": "Illegal material distributed online", "date": "2024-03-06", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Bomb Threat", "location": "Plano", "description": "Threat called into office building", "date": "2024-02-05", "time": "09:15", "criminal_name": "Unknown" },
  { "crime_type": "Elder Abuse", "location": "Bakersfield", "description": "Nursing home resident financially exploited", "date": "2024-01-24", "time": "Unknown", "criminal_name": "Rachel Thompson" },
  { "crime_type": "Illegal Waste Disposal", "location": "Laredo", "description": "Industrial chemicals dumped in empty lot", "date": "2024-03-02", "time": "04:05", "criminal_name": "Unknown" },
  { "crime_type": "Road Rage", "location": "Akron", "description": "Driver brandished weapon after being cut off", "date": "2024-02-05", "time": "17:10", "criminal_name": "Unknown" },
  { "crime_type": "Wire Fraud", "location": "Yonkers", "description": "Funds diverted from business wire transfer", "date": "2024-01-01", "time": "Unknown", "criminal_name": "Walter Ferguson" }
]

### DB 5

In [ ]:
# 📝 New Crime Report Dataset
crime_reports = [
  { "crime_type": "Identity Theft", "location": "Seattle", "description": "Credit cards opened in victim's name", "date": "2024-05-17", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Minneapolis", "description": "Forced entry through garage door", "date": "2024-06-02", "time": "02:15", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Denver", "description": "Vehicle stolen from shopping mall parking lot", "date": "2024-05-03", "time": "15:40", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Los Angeles", "description": "Altercation at nightclub entrance", "date": "2024-06-12", "time": "23:45", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Fraud", "location": "Phoenix", "description": "Elderly victims solicited for fake charity", "date": "2024-05-29", "time": "14:20", "criminal_name": "Victoria Stephens" },
  { "crime_type": "Shoplifting", "location": "Portland", "description": "High-end electronics concealed in backpack", "date": "2024-06-08", "time": "16:35", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Baltimore", "description": "Graffiti on public library building", "date": "2024-05-11", "time": "01:25", "criminal_name": "Unknown" },
  { "crime_type": "Drug Possession", "location": "New Orleans", "description": "Illegal substances found during traffic stop", "date": "2024-06-17", "time": "20:10", "criminal_name": "Unknown" },
  { "crime_type": "DUI", "location": "San Diego", "description": "Driver went wrong way on one-way street", "date": "2024-05-22", "time": "00:15", "criminal_name": "Timothy Wright" },
  { "crime_type": "Cyber Attack", "location": "Boston", "description": "Municipal government systems breached", "date": "2024-06-05", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Kansas City", "description": "Convenience store held up at closing time", "date": "2024-05-07", "time": "22:50", "criminal_name": "Jacob Torres" },
  { "crime_type": "Trespassing", "location": "Sacramento", "description": "Unauthorized access to construction site", "date": "2024-06-21", "time": "03:30", "criminal_name": "Unknown" },
  { "crime_type": "Embezzlement", "location": "Chicago", "description": "Nonprofit funds diverted to personal accounts", "date": "2024-05-14", "time": "Unknown", "criminal_name": "Rebecca Collins" },
  { "crime_type": "Identity Theft", "location": "St. Louis", "description": "Medical insurance used fraudulently", "date": "2024-06-09", "time": "10:45", "criminal_name": "Unknown" },
  { "crime_type": "Stalking", "location": "Nashville", "description": "Victim followed home from workplace repeatedly", "date": "2024-05-26", "time": "18:20", "criminal_name": "Unknown" },
  { "crime_type": "Arson", "location": "Atlanta", "description": "Abandoned building set on fire", "date": "2024-06-14", "time": "02:05", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Weapon Possession", "location": "Houston", "description": "Firearm discovered during apartment search", "date": "2024-05-19", "time": "15:15", "criminal_name": "Unknown" },
  { "crime_type": "Blackmail", "location": "Philadelphia", "description": "Victim threatened with release of private information", "date": "2024-06-03", "time": "Unknown", "criminal_name": "Andrea Parker" },
  { "crime_type": "Counterfeiting", "location": "Miami", "description": "Fake currency used at multiple retail locations", "date": "2024-05-28", "time": "13:10", "criminal_name": "Unknown" },
  { "crime_type": "Child Endangerment", "location": "Detroit", "description": "Children found in vehicle during parent's arrest", "date": "2024-06-25", "time": "09:35", "criminal_name": "Unknown" },
  { "crime_type": "Extortion", "location": "San Francisco", "description": "Restaurant owner threatened for protection money", "date": "2024-05-01", "time": "21:05", "criminal_name": "Marcus Henderson" },
  { "crime_type": "Hit and Run", "location": "Cleveland", "description": "Cyclist struck on bike lane", "date": "2024-06-18", "time": "07:30", "criminal_name": "Unknown" },
  { "crime_type": "Public Intoxication", "location": "Austin", "description": "Disorderly conduct at public park", "date": "2024-05-31", "time": "16:55", "criminal_name": "Unknown" },
  { "crime_type": "Forgery", "location": "Tampa", "description": "Check altered before deposit", "date": "2024-06-07", "time": "11:20", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Dumping", "location": "Pittsburgh", "description": "Hazardous materials left near water source", "date": "2024-05-23", "time": "04:50", "criminal_name": "Unknown" },
  { "crime_type": "Human Trafficking", "location": "Las Vegas", "description": "Victims transported across state lines", "date": "2024-06-11", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Money Laundering", "location": "Dallas", "description": "Restaurant business used to clean illegal funds", "date": "2024-05-04", "time": "Unknown", "criminal_name": "Nathaniel Rogers" },
  { "crime_type": "Kidnapping", "location": "Tucson", "description": "Short-term abduction during custody dispute", "date": "2024-06-29", "time": "15:55", "criminal_name": "Melissa Turner" },
  { "crime_type": "Harassment", "location": "Raleigh", "description": "Repeated threatening phone calls", "date": "2024-05-12", "time": "22:40", "criminal_name": "Unknown" },
  { "crime_type": "Tax Evasion", "location": "San Antonio", "description": "Unreported income from side business", "date": "2024-06-16", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Gambling", "location": "New York", "description": "Underground poker operation discovered", "date": "2024-05-27", "time": "23:30", "criminal_name": "Samuel Brooks" },
  { "crime_type": "Credit Card Fraud", "location": "Charlotte", "description": "Fraudulent purchases made online", "date": "2024-06-22", "time": "12:15", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Oakland", "description": "Bar fight resulting in serious injury", "date": "2024-05-08", "time": "00:50", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Drug Distribution", "location": "Louisville", "description": "Dealer caught in park near school", "date": "2024-06-01", "time": "16:40", "criminal_name": "Derrick Powell" },
  { "crime_type": "Identity Theft", "location": "Cincinnati", "description": "Utility accounts opened with stolen information", "date": "2024-05-16", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Milwaukee", "description": "Online advertisement leading to hotel sting", "date": "2024-06-13", "time": "20:25", "criminal_name": "Unknown" },
  { "crime_type": "Bribery", "location": "Columbus", "description": "City official accepting payment for permits", "date": "2024-05-20", "time": "10:05", "criminal_name": "Harold Mitchell" },
  { "crime_type": "Burglary", "location": "Indianapolis", "description": "Business break-in through roof access", "date": "2024-06-24", "time": "03:45", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Washington DC", "description": "Keyless entry vehicle stolen from driveway", "date": "2024-05-09", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Insider Trading", "location": "Jersey City", "description": "Stock purchased before public announcement", "date": "2024-06-06", "time": "14:30", "criminal_name": "Lawrence Grant" },
  { "crime_type": "Child Abuse", "location": "Albuquerque", "description": "School reported suspicious injuries", "date": "2024-05-25", "time": "09:15", "criminal_name": "Unknown" },
  { "crime_type": "Grand Theft", "location": "Memphis", "description": "High-value equipment stolen from warehouse", "date": "2024-06-19", "time": "01:10", "criminal_name": "Unknown" },
  { "crime_type": "Domestic Violence", "location": "Orlando", "description": "Neighbor reported disturbance", "date": "2024-05-18", "time": "22:35", "criminal_name": "Unknown" },
  { "crime_type": "Impersonation", "location": "Oklahoma City", "description": "Individual claimed to be government official", "date": "2024-06-27", "time": "11:50", "criminal_name": "Unknown" },
  { "crime_type": "Cyberbullying", "location": "Virginia Beach", "description": "Student targeted through social media", "date": "2024-05-06", "time": "17:20", "criminal_name": "Unknown" },
  { "crime_type": "Insurance Fraud", "location": "Fort Worth", "description": "False injury claim after minor accident", "date": "2024-06-10", "time": "13:45", "criminal_name": "Gabrielle Foster" },
  { "crime_type": "Shoplifting", "location": "Jacksonville", "description": "Items concealed in shopping cart bottom", "date": "2024-05-30", "time": "18:05", "criminal_name": "Unknown" },
  { "crime_type": "Reckless Driving", "location": "El Paso", "description": "Excessive speeding in school zone", "date": "2024-06-15", "time": "08:20", "criminal_name": "Unknown" },
  { "crime_type": "Animal Cruelty", "location": "Buffalo", "description": "Pets abandoned in foreclosed home", "date": "2024-05-21", "time": "16:10", "criminal_name": "Unknown" },
  { "crime_type": "Carjacking", "location": "Omaha", "description": "Driver threatened at gas station", "date": "2024-06-28", "time": "22:15", "criminal_name": "Unknown" },
  { "crime_type": "Online Scam", "location": "Long Beach", "description": "Fraudulent rental listings collecting deposits", "date": "2024-05-13", "time": "Unknown", "criminal_name": "Jessica Coleman" },
  { "crime_type": "Vandalism", "location": "Tulsa", "description": "Multiple cars damaged in parking garage", "date": "2024-06-23", "time": "02:40", "criminal_name": "Unknown" },
  { "crime_type": "Ponzi Scheme", "location": "Wichita", "description": "Fraudulent investment targeting senior citizens", "date": "2024-05-02", "time": "Unknown", "criminal_name": "Douglas Price" },
  { "crime_type": "Theft of Services", "location": "Fresno", "description": "Restaurant patrons left without paying", "date": "2024-06-04", "time": "19:30", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Madison", "description": "Fake invoices sent to businesses", "date": "2024-05-24", "time": "11:05", "criminal_name": "Victoria Stephens" },
  { "crime_type": "Unlawful Possession of Firearm", "location": "Anaheim", "description": "Concealed weapon found during event security", "date": "2024-06-20", "time": "19:50", "criminal_name": "Aaron Carter" },
  { "crime_type": "Identity Theft", "location": "Richmond", "description": "Fraudulent loan applications submitted", "date": "2024-05-15", "time": "14:55", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Baton Rouge", "description": "Gas station clerk threatened at knifepoint", "date": "2024-06-26", "time": "00:25", "criminal_name": "Jacob Torres" },
  { "crime_type": "Loitering", "location": "Greensboro", "description": "Group refusing to disperse from private property", "date": "2024-05-10", "time": "21:15", "criminal_name": "Multiple Unknown" },
  { "crime_type": "DUI", "location": "Spokane", "description": "Driver found unconscious at intersection", "date": "2024-06-30", "time": "01:35", "criminal_name": "Christina Morris" },
  { "crime_type": "Check Fraud", "location": "Norfolk", "description": "Altered business checks deposited remotely", "date": "2024-05-05", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Child Neglect", "location": "Santa Ana", "description": "Children found wandering near busy street", "date": "2024-06-08", "time": "11:30", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Providence", "description": "Online advertisement leading to arrest", "date": "2024-05-29", "time": "21:40", "criminal_name": "Unknown" },
  { "crime_type": "Corporate Espionage", "location": "Irvine", "description": "Trade secrets stolen by former employee", "date": "2024-06-12", "time": "Unknown", "criminal_name": "Wesley Hammond" },
  { "crime_type": "Assault", "location": "Toledo", "description": "Victim attacked leaving sporting event", "date": "2024-05-27", "time": "22:20", "criminal_name": "Unknown" },
  { "crime_type": "Perjury", "location": "Durham", "description": "Witness provided false testimony", "date": "2024-06-03", "time": "14:05", "criminal_name": "Unknown" },
  { "crime_type": "Drug Trafficking", "location": "Aurora", "description": "Shipment intercepted in mail system", "date": "2024-05-11", "time": "09:55", "criminal_name": "Derrick Powell" },
  { "crime_type": "Auto Theft", "location": "Boise", "description": "Multiple vehicles taken from apartment complex", "date": "2024-06-17", "time": "03:25", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Newark", "description": "Office break-in with multiple items stolen", "date": "2024-05-23", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Harassment", "location": "Anchorage", "description": "Repeated unwanted contact after warnings", "date": "2024-06-09", "time": "16:50", "criminal_name": "Eric Sullivan" },
  { "crime_type": "Identity Theft", "location": "St. Paul", "description": "Victim's information used for employment", "date": "2024-05-16", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Trespassing", "location": "Rochester", "description": "Individual found in restricted area", "date": "2024-06-22", "time": "04:15", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Tacoma", "description": "School property damaged overnight", "date": "2024-05-19", "time": "00:40", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Modesto", "description": "Home repair scam targeting elderly", "date": "2024-06-05", "time": "Unknown", "criminal_name": "Patrick Walters" },
  { "crime_type": "Shoplifting", "location": "Chandler", "description": "Cosmetics concealed in clothing", "date": "2024-05-31", "time": "15:35", "criminal_name": "Unknown" },
  { "crime_type": "Cyber Attack", "location": "Reno", "description": "Hospital systems compromised", "date": "2024-06-28", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Drug Manufacturing", "location": "Glendale", "description": "Operation discovered in residential home", "date": "2024-05-07", "time": "10:20", "criminal_name": "Unknown" },
  { "crime_type": "Criminal Mischief", "location": "Fort Wayne", "description": "Park playground equipment damaged", "date": "2024-06-14", "time": "23:05", "criminal_name": "Unknown" },
  { "crime_type": "Disorderly Conduct", "location": "Chula Vista", "description": "Disruption at government meeting", "date": "2024-05-25", "time": "13:50", "criminal_name": "Martin Reynolds" },
  { "crime_type": "Larceny", "location": "Worcester", "description": "Package stolen from doorstep", "date": "2024-06-19", "time": "15:10", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Immigration", "location": "Hialeah", "description": "Fraudulent documents used for employment", "date": "2024-05-03", "time": "08:30", "criminal_name": "Unknown" },
  { "crime_type": "Homicide", "location": "St. Petersburg", "description": "Body discovered in abandoned building", "date": "2024-06-25", "time": "05:45", "criminal_name": "Unknown" },
  { "crime_type": "Identity Theft", "location": "Lincoln", "description": "False tax return filed in victim's name", "date": "2024-05-14", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Breaking and Entering", "location": "Lubbock", "description": "Business entered through broken window", "date": "2024-06-07", "time": "02:35", "criminal_name": "Unknown" },
  { "crime_type": "Copyright Infringement", "location": "Stockton", "description": "Illegal streaming service operation", "date": "2024-05-22", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Fare Evasion", "location": "Jersey City", "description": "Multiple people avoiding payment", "date": "2024-06-11", "time": "07:15", "criminal_name": "Multiple Unknown" },
  { "crime_type": "Assault", "location": "Buffalo", "description": "Fight at public sports venue", "date": "2024-05-09", "time": "20:45", "criminal_name": "Franklin Jenkins" },
  { "crime_type": "Credit Card Theft", "location": "Gilbert", "description": "Card information stolen during transaction", "date": "2024-06-15", "time": "12:10", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Firearms Sale", "location": "Chesapeake", "description": "Transaction in parking lot reported", "date": "2024-05-28", "time": "17:30", "criminal_name": "Unknown" },
  { "crime_type": "Pickpocketing", "location": "Winston-Salem", "description": "Wallet stolen during street festival", "date": "2024-06-02", "time": "14:40", "criminal_name": "Unknown" },
  { "crime_type": "Child Pornography", "location": "Irving", "description": "Material discovered during device inspection", "date": "2024-05-18", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Bomb Threat", "location": "Scottsdale", "description": "School evacuation after call received", "date": "2024-06-21", "time": "10:35", "criminal_name": "Unknown" },
  { "crime_type": "Elder Abuse", "location": "Fremont", "description": "Family member misusing finances", "date": "2024-05-04", "time": "Unknown", "criminal_name": "Rachel Thompson" },
  { "crime_type": "Illegal Waste Disposal", "location": "Montgomery", "description": "Medical waste found in residential trash", "date": "2024-06-13", "time": "06:25", "criminal_name": "Unknown" },
  { "crime_type": "Road Rage", "location": "Bakersfield", "description": "Drivers exiting vehicles during altercation", "date": "2024-05-26", "time": "18:15", "criminal_name": "Unknown" },
  { "crime_type": "Wire Fraud", "location": "Des Moines", "description": "Money transfer to fraudulent account", "date": "2024-06-29", "time": "Unknown", "criminal_name": "Walter Ferguson" }
]

### DB 6

In [ ]:
# 📝 New Crime Report Dataset
crime_reports = [
  { "crime_type": "Identity Theft", "location": "Chicago", "description": "Bank accounts opened without authorization", "date": "2024-09-12", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Tampa", "description": "Entry through unlocked window", "date": "2024-08-28", "time": "03:20", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Austin", "description": "Vehicle stolen from airport long-term parking", "date": "2024-07-18", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Boston", "description": "Fight outside sports arena", "date": "2024-09-03", "time": "22:15", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Fraud", "location": "Atlanta", "description": "Fake investment opportunity targeting retirees", "date": "2024-08-15", "time": "10:30", "criminal_name": "Victoria Stephens" },
  { "crime_type": "Shoplifting", "location": "San Francisco", "description": "Jewelry hidden in clothing", "date": "2024-09-07", "time": "14:45", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Minneapolis", "description": "Damage to public park equipment", "date": "2024-07-29", "time": "00:55", "criminal_name": "Unknown" },
  { "crime_type": "Drug Possession", "location": "Miami", "description": "Substances found during routine inspection", "date": "2024-08-30", "time": "18:40", "criminal_name": "Unknown" },
  { "crime_type": "DUI", "location": "Nashville", "description": "Driver weaving between lanes", "date": "2024-09-10", "time": "01:25", "criminal_name": "Christina Morris" },
  { "crime_type": "Cyber Attack", "location": "Seattle", "description": "School district network compromised", "date": "2024-07-21", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Houston", "description": "Bank customer followed and robbed", "date": "2024-08-06", "time": "14:10", "criminal_name": "Jacob Torres" },
  { "crime_type": "Trespassing", "location": "Los Angeles", "description": "Individual found in private residence", "date": "2024-09-15", "time": "05:20", "criminal_name": "Unknown" },
  { "crime_type": "Embezzlement", "location": "Denver", "description": "Company funds diverted through fake invoices", "date": "2024-07-12", "time": "Unknown", "criminal_name": "Rebecca Collins" },
  { "crime_type": "Identity Theft", "location": "Portland", "description": "Social security number used for employment", "date": "2024-08-22", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Stalking", "location": "Dallas", "description": "Victim receiving repeated unwanted contact", "date": "2024-09-01", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Arson", "location": "Phoenix", "description": "Vehicle set on fire in driveway", "date": "2024-08-11", "time": "03:35", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Weapon Possession", "location": "Detroit", "description": "Concealed weapon at public event", "date": "2024-07-25", "time": "19:45", "criminal_name": "Aaron Carter" },
  { "crime_type": "Blackmail", "location": "New York", "description": "Business owner threatened over past mistakes", "date": "2024-08-17", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Counterfeiting", "location": "San Diego", "description": "Fake bills used at multiple businesses", "date": "2024-09-05", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Child Endangerment", "location": "St. Louis", "description": "Young children left unattended in vehicle", "date": "2024-08-02", "time": "15:30", "criminal_name": "Unknown" },
  { "crime_type": "Extortion", "location": "Charlotte", "description": "Business threatened with false reviews", "date": "2024-07-08", "time": "Unknown", "criminal_name": "Marcus Henderson" },
  { "crime_type": "Hit and Run", "location": "Pittsburgh", "description": "Pedestrian struck in crosswalk", "date": "2024-08-24", "time": "18:05", "criminal_name": "Unknown" },
  { "crime_type": "Public Intoxication", "location": "New Orleans", "description": "Belligerent behavior in tourist area", "date": "2024-09-11", "time": "22:40", "criminal_name": "Unknown" },
  { "crime_type": "Forgery", "location": "Indianapolis", "description": "Signature forged on legal documents", "date": "2024-07-31", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Dumping", "location": "Kansas City", "description": "Construction waste left in public area", "date": "2024-08-09", "time": "02:15", "criminal_name": "Unknown" },
  { "crime_type": "Human Trafficking", "location": "Reno", "description": "Labor trafficking in restaurant industry", "date": "2024-09-02", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Money Laundering", "location": "Sacramento", "description": "Cash-intensive business used to hide funds", "date": "2024-07-16", "time": "Unknown", "criminal_name": "Nathaniel Rogers" },
  { "crime_type": "Kidnapping", "location": "Cincinnati", "description": "Child taken by non-custodial parent", "date": "2024-08-26", "time": "14:25", "criminal_name": "Melissa Turner" },
  { "crime_type": "Harassment", "location": "Louisville", "description": "Unwanted messages sent after restraining order", "date": "2024-09-08", "time": "Various", "criminal_name": "Eric Sullivan" },
  { "crime_type": "Tax Evasion", "location": "Oklahoma City", "description": "Business owner hiding cash transactions", "date": "2024-07-27", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Gambling", "location": "Baltimore", "description": "Betting operation in private club", "date": "2024-08-13", "time": "21:50", "criminal_name": "Samuel Brooks" },
  { "crime_type": "Credit Card Fraud", "location": "Columbus", "description": "Skimming device installed at gas station", "date": "2024-09-17", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Assault", "location": "Memphis", "description": "Physical altercation at concert", "date": "2024-07-23", "time": "21:35", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Drug Distribution", "location": "Washington DC", "description": "Drugs sold near community center", "date": "2024-08-04", "time": "17:15", "criminal_name": "Derrick Powell" },
  { "crime_type": "Identity Theft", "location": "Orlando", "description": "Victim's information used for loan application", "date": "2024-09-19", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Tucson", "description": "Services offered through massage business", "date": "2024-07-14", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Bribery", "location": "Albuquerque", "description": "Contractor offering payment for contract approval", "date": "2024-08-20", "time": "13:40", "criminal_name": "Harold Mitchell" },
  { "crime_type": "Burglary", "location": "Cleveland", "description": "Forced entry through rear door", "date": "2024-09-06", "time": "04:15", "criminal_name": "Unknown" },
  { "crime_type": "Auto Theft", "location": "Jacksonville", "description": "Motorcycle stolen from parking garage", "date": "2024-07-19", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Insider Trading", "location": "Philadelphia", "description": "Shares sold before negative news release", "date": "2024-08-18", "time": "09:30", "criminal_name": "Lawrence Grant" },
  { "crime_type": "Child Abuse", "location": "Fort Worth", "description": "Medical professional reported concerning injuries", "date": "2024-09-14", "time": "11:05", "criminal_name": "Unknown" },
  { "crime_type": "Grand Theft", "location": "Long Beach", "description": "Art stolen from gallery exhibition", "date": "2024-07-30", "time": "00:20", "criminal_name": "Unknown" },
  { "crime_type": "Domestic Violence", "location": "El Paso", "description": "Victim sought medical attention after assault", "date": "2024-08-08", "time": "23:15", "criminal_name": "Unknown" },
  { "crime_type": "Impersonation", "location": "Milwaukee", "description": "Individual posed as utility worker", "date": "2024-09-04", "time": "13:50", "criminal_name": "Unknown" },
  { "crime_type": "Cyberbullying", "location": "Omaha", "description": "Harassment through gaming platform", "date": "2024-07-22", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Insurance Fraud", "location": "Oakland", "description": "Staged accident for insurance claim", "date": "2024-08-16", "time": "11:30", "criminal_name": "Gabrielle Foster" },
  { "crime_type": "Shoplifting", "location": "Tulsa", "description": "Multiple items concealed in stroller", "date": "2024-09-13", "time": "16:20", "criminal_name": "Unknown" },
  { "crime_type": "Reckless Driving", "location": "Fresno", "description": "Street racing on public roads", "date": "2024-07-15", "time": "00:45", "criminal_name": "Unknown" },
  { "crime_type": "Animal Cruelty", "location": "Richmond", "description": "Dogs found in poor living conditions", "date": "2024-08-27", "time": "10:15", "criminal_name": "Unknown" },
  { "crime_type": "Carjacking", "location": "Virginia Beach", "description": "Driver forced from vehicle at intersection", "date": "2024-09-09", "time": "23:40", "criminal_name": "Unknown" },
  { "crime_type": "Online Scam", "location": "Honolulu", "description": "Fake tech support extracting payment", "date": "2024-07-10", "time": "Unknown", "criminal_name": "Jessica Coleman" },
  { "crime_type": "Vandalism", "location": "Baton Rouge", "description": "Windows broken at local business", "date": "2024-08-19", "time": "01:50", "criminal_name": "Unknown" },
  { "crime_type": "Ponzi Scheme", "location": "Rochester", "description": "Investment scam promising unrealistic returns", "date": "2024-09-16", "time": "Unknown", "criminal_name": "Douglas Price" },
  { "crime_type": "Theft of Services", "location": "Wichita", "description": "Hotel rooms used without payment", "date": "2024-07-24", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Chandler", "description": "Contractors paid without work performed", "date": "2024-08-05", "time": "Unknown", "criminal_name": "Victoria Stephens" },
  { "crime_type": "Unlawful Possession of Firearm", "location": "Raleigh", "description": "Prohibited person found with weapon", "date": "2024-09-20", "time": "15:40", "criminal_name": "Unknown" },
  { "crime_type": "Identity Theft", "location": "Jersey City", "description": "Phone accounts opened with stolen identity", "date": "2024-07-11", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Robbery", "location": "Spokane", "description": "Pharmacy held up for prescription drugs", "date": "2024-08-21", "time": "20:05", "criminal_name": "Jacob Torres" },
  { "crime_type": "Loitering", "location": "Madison", "description": "Individuals refusing to leave bank lobby", "date": "2024-09-18", "time": "16:30", "criminal_name": "Multiple Unknown" },
  { "crime_type": "DUI", "location": "Aurora", "description": "Driver collided with parked vehicles", "date": "2024-07-28", "time": "02:10", "criminal_name": "Timothy Wright" },
  { "crime_type": "Check Fraud", "location": "Santa Ana", "description": "Counterfeit checks passed at multiple locations", "date": "2024-08-10", "time": "Various", "criminal_name": "Unknown" },
  { "crime_type": "Child Neglect", "location": "Buffalo", "description": "Young child left home alone overnight", "date": "2024-09-22", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Prostitution", "location": "Riverside", "description": "Services advertised through social media", "date": "2024-07-13", "time": "19:20", "criminal_name": "Unknown" },
  { "crime_type": "Corporate Espionage", "location": "Irvine", "description": "Competitor's data stolen by new employee", "date": "2024-08-23", "time": "Unknown", "criminal_name": "Wesley Hammond" },
  { "crime_type": "Assault", "location": "St. Paul", "description": "Victim assaulted in parking structure", "date": "2024-09-01", "time": "01:15", "criminal_name": "Franklin Jenkins" },
  { "crime_type": "Perjury", "location": "Anaheim", "description": "False statement given under oath", "date": "2024-07-17", "time": "10:40", "criminal_name": "Unknown" },
  { "crime_type": "Drug Trafficking", "location": "Lexington", "description": "Package containing drugs intercepted", "date": "2024-08-31", "time": "Unknown", "criminal_name": "Derrick Powell" },
  { "crime_type": "Auto Theft", "location": "Corpus Christi", "description": "Rental car not returned after due date", "date": "2024-09-21", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Burglary", "location": "Stockton", "description": "Storage unit broken into overnight", "date": "2024-07-26", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Harassment", "location": "Plano", "description": "Coworker making threatening statements", "date": "2024-08-14", "time": "14:15", "criminal_name": "Unknown" },
  { "crime_type": "Identity Theft", "location": "Newark", "description": "Mail stolen from residential mailboxes", "date": "2024-09-23", "time": "Early morning", "criminal_name": "Unknown" },
  { "crime_type": "Trespassing", "location": "Anchorage", "description": "Individual sleeping in commercial building", "date": "2024-07-07", "time": "05:35", "criminal_name": "Unknown" },
  { "crime_type": "Vandalism", "location": "Henderson", "description": "Graffiti on highway overpass", "date": "2024-08-29", "time": "23:50", "criminal_name": "Unknown" },
  { "crime_type": "Fraud", "location": "Bakersfield", "description": "Roof repair scam after storm damage", "date": "2024-09-25", "time": "14:00", "criminal_name": "Patrick Walters" },
  { "crime_type": "Shoplifting", "location": "Fort Wayne", "description": "Security tags removed in fitting room", "date": "2024-07-20", "time": "17:25", "criminal_name": "Unknown" },
  { "crime_type": "Cyber Attack", "location": "Durham", "description": "Small business ransomware attack", "date": "2024-08-03", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Drug Manufacturing", "location": "Modesto", "description": "Lab discovered in storage facility", "date": "2024-09-27", "time": "11:15", "criminal_name": "Unknown" },
  { "crime_type": "Criminal Mischief", "location": "Scottsdale", "description": "HOA property deliberately damaged", "date": "2024-07-09", "time": "20:30", "criminal_name": "Unknown" },
  { "crime_type": "Disorderly Conduct", "location": "Norfolk", "description": "Individual causing scene at hospital", "date": "2024-08-25", "time": "09:40", "criminal_name": "Martin Reynolds" },
  { "crime_type": "Larceny", "location": "Birmingham", "description": "Wallet stolen from gym locker", "date": "2024-09-29", "time": "18:50", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Immigration", "location": "Chula Vista", "description": "Falsified documents for employment", "date": "2024-07-05", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Homicide", "location": "Greensboro", "description": "Shooting at apartment complex", "date": "2024-08-07", "time": "22:30", "criminal_name": "Unknown" },
  { "crime_type": "Identity Theft", "location": "Gilbert", "description": "Child's identity used for credit accounts", "date": "2024-09-24", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Breaking and Entering", "location": "Garland", "description": "Vacation home broken into", "date": "2024-07-03", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Copyright Infringement", "location": "Chesapeake", "description": "Unauthorized reproduction of software", "date": "2024-08-12", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Fare Evasion", "location": "Lincoln", "description": "Group bypassing transit payment system", "date": "2024-09-30", "time": "16:45", "criminal_name": "Multiple Unknown" },
  { "crime_type": "Assault", "location": "Worcester", "description": "Workplace violence incident", "date": "2024-07-01", "time": "09:25", "criminal_name": "Lucas Bennett" },
  { "crime_type": "Credit Card Theft", "location": "Winston-Salem", "description": "Cards stolen from gym locker", "date": "2024-08-01", "time": "17:55", "criminal_name": "Unknown" },
  { "crime_type": "Illegal Firearms Sale", "location": "Laredo", "description": "Undercover operation at gun show", "date": "2024-09-28", "time": "13:05", "criminal_name": "Unknown" },
  { "crime_type": "Pickpocketing", "location": "Glendale", "description": "Tourist targeted at attraction", "date": "2024-07-04", "time": "15:20", "criminal_name": "Unknown" },
  { "crime_type": "Child Pornography", "location": "Irving", "description": "Images found during computer repair", "date": "2024-08-30", "time": "Unknown", "criminal_name": "Unknown" },
  { "crime_type": "Bomb Threat", "location": "Lubbock", "description": "Corporate office evacuated after call", "date": "2024-09-26", "time": "08:45", "criminal_name": "Unknown" },
  { "crime_type": "Elder Abuse", "location": "Hialeah", "description": "Caregiver stealing from elderly patient", "date": "2024-07-06", "time": "Unknown", "criminal_name": "Rachel Thompson" },
  { "crime_type": "Illegal Waste Disposal", "location": "Tacoma", "description": "Chemicals dumped in vacant lot", "date": "2024-08-20", "time": "04:10", "criminal_name": "Unknown" },
  { "crime_type": "Road Rage", "location": "Des Moines", "description": "Driver threatened with weapon", "date": "2024-09-12", "time": "17:40", "criminal_name": "Unknown" },
  { "crime_type": "Wire Fraud", "location": "Boise", "description": "Business email compromise scheme", "date": "2024-07-02", "time": "Unknown", "criminal_name": "Walter Ferguson" }
]

### Keywords

In [2]:
keywords = [
  "Identity Theft",
  "Burglary",
  "Auto Theft",
  "Assault",
  "Fraud",
  "Shoplifting",
  "Vandalism",
  "Drug Possession",
  "DUI",
  "Cyber Attack",
  "Robbery",
  "Trespassing",
  "Embezzlement",
  "Stalking",
  "Arson",
  "Illegal Weapon Possession",
  "Blackmail",
  "Counterfeiting",
  "Child Endangerment",
  "Extortion",
  "Hit and Run",
  "Public Intoxication",
  "Forgery",
  "Illegal Dumping",
  "Human Trafficking",
  "Money Laundering",
  "Kidnapping",
  "Harassment",
  "Tax Evasion",
  "Illegal Gambling",
  "Credit Card Fraud",
  "Drug Distribution",
  "Prostitution",
  "Bribery",
  "Insider Trading",
  "Child Abuse",
  "Grand Theft",
  "Domestic Violence",
  "Impersonation",
  "Cyberbullying",
  "Insurance Fraud",
  "Reckless Driving",
  "Animal Cruelty",
  "Carjacking",
  "Online Scam",
  "Ponzi Scheme",
  "Theft of Services",
  "Unlawful Possession of Firearm",
  "Loitering",
  "Check Fraud",
  "Child Neglect",
  "Corporate Espionage",
  "Perjury",
  "Drug Trafficking",
  "Criminal Mischief",
  "Disorderly Conduct",
  "Larceny",
  "Illegal Immigration",
  "Homicide",
  "Breaking and Entering",
  "Copyright Infringement",
  "Fare Evasion",
  "Credit Card Theft",
  "Illegal Firearms Sale",
  "Pickpocketing",
  "Child Pornography",
  "Bomb Threat",
  "Elder Abuse",
  "Illegal Waste Disposal",
  "Road Rage",
  "Wire Fraud",
  "Illegal Drug Manufacturing"
]

### CKKS Key Generation

The below code has to run once only.

Before that create a table 
```sql
 CREATE TABLE ckks_keys (
     id SERIAL PRIMARY KEY,
     name TEXT UNIQUE,              -- Optional: name tag like 'main_ckks_key'
     context BYTEA NOT NULL,        -- Stores the full serialized CKKS context
     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
 );
```
Inside the database where you have other keys. 

In [ ]:
# 🔐 Step 2: Create new CKKS context
# Create a new CKKS context with suitable parameters
poly_modulus_degree = 8192  # Example degree (adjust if needed)
coeff_mod_bit_sizes = [60, 40, 60]  # Example bit sizes (adjust if needed)

ckks_context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree, coeff_mod_bit_sizes)

# Set the context's secret key for encryption (this is a public key, the secret key should be available)
ckks_context.generate_galois_keys()

In [ ]:
import tenseal as ts
import psycopg2

# Assuming ckks_context already exists
ckks_serialized = ckks_context.serialize(
    save_public_key=True,
    save_secret_key=True,
    save_galois_keys=True,
    save_relin_keys=True
)

conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

# Insert into the new table
cur.execute("""
    INSERT INTO ckks_keys (name, context)
    VALUES (%s, %s)
    ON CONFLICT (name) DO UPDATE SET context = EXCLUDED.context
""", ('main_ckks_key', psycopg2.Binary(ckks_serialized)))

conn.commit()
cur.close()
conn.close()

print("✅ CKKS context stored in 'ckks_keys' table.")


Then Start Insertion.

In [ ]:
import tenseal as ts
import psycopg2
import base64

# 🔐 Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)

newdb = psycopg2.connect(
    dbname='Database 6',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)

cur = conn.cursor()
newcur = newdb.cursor()

# 🔐 Step 1: Load BFV context from the database (for encrypting other fields)
cur.execute("SELECT public_key FROM encryption_keys WHERE id = %s", (4,))  # Adjust ID if needed
row = cur.fetchone()
if not row:
    raise Exception("No public key found with id = 3")

public_key_bytes = bytes(row[0])
# Load BFV context using the public key
bfv_context = ts.context_from(public_key_bytes)



# ✨ Vectorize crime type
def vectorize_crime_type(crime_type, keyword_list):
    vector = [0.0] * len(keyword_list)
    crime_type = crime_type.lower()
    for i, word in enumerate(keyword_list):
        if word.lower() in crime_type:
            vector[i] = 1.0
    return vector

# 🔒 Fixed-length encryption (BFV) for other fields
def encrypt_string_bfv(context, text, max_length=30):
    encoded = [ord(c) for c in text[:max_length]]
    encoded += [0] * (max_length - len(encoded))
    bfv_context = ts.bfv_vector(context, encoded)
    return bfv_context.serialize()

# 🔁 Encrypt and insert each report
for crime_report in crime_reports:
    # CKKS: Encrypt vectorized crime type
    vectorized = vectorize_crime_type(crime_report['crime_type'], keywords)
    
    # Ensure the vector is of type float (CKKS expects floats)
    vectorized = [float(v) for v in vectorized]
    
    # Encrypt with CKKS context
    ckks_vector = ts.ckks_vector(ckks_context, vectorized, scale=2 ** 40)
    encrypted_ckks = ckks_vector.serialize()

    # BFV: Encrypt other fields (location, description, etc.)
    encrypted_data = {
        key: encrypt_string_bfv(bfv_context, value)
        for key, value in crime_report.items() if key != 'crime_type'
    }

    # INSERT into encrypted DB
    newcur.execute("""
        INSERT INTO encrypted_crime_reports (
            encryption_key_id, crime_type, location, description,
            date, time, criminal_name, encrypted_crime_type
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        4,
        psycopg2.Binary(encrypted_ckks),  # vectorized crime_type (encrypted with CKKS)
        psycopg2.Binary(encrypted_data["location"]),
        psycopg2.Binary(encrypted_data["description"]),
        psycopg2.Binary(encrypted_data["date"]),
        psycopg2.Binary(encrypted_data["time"]),
        psycopg2.Binary(encrypted_data["criminal_name"]),
        psycopg2.Binary(encrypted_ckks)  # storing the encrypted CKKS vector again for searching
    ))

# ✅ Finalize
conn.commit()
newdb.commit()
cur.close()
newcur.close()
conn.close()
newdb.close()

print("✅ Encrypted crime reports inserted using hybrid encryption (CKKS for crime type, BFV for others).")


## Decryption

Load the Ckks Key and then run the code for searching.
To get the result it is necessary that you make a accurate query without spelling mistakes.
Also this is going to search for `crime_type`. 

In [3]:
import tenseal as ts
import psycopg2


conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
cur = conn.cursor()

cur.execute("SELECT context FROM ckks_keys WHERE name = %s", ('main_ckks_key',))
row = cur.fetchone()
if not row:
    raise Exception("❌ CKKS context not found with name = 'main_ckks_key'")

pip_context = ts.context_from(bytes(row[0]))

cur.close()
conn.close()

print("✅ CKKS context loaded from 'ckks_keys' table.")


✅ CKKS context loaded from 'ckks_keys' table.


In [7]:
import tenseal as ts
import psycopg2

# 🔐 Connect to NewDatabase for encryption keys
key_db_conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
key_db_cursor = key_db_conn.cursor()

# 🔐 Vectorize the search term
def encode_search_term(search_term, known_crime_types):
    return [1 if search_term.lower() == crime.lower() else 0 for crime in known_crime_types]

# 🔐 Decrypt CKKS vector
def decrypt_ckks_vector(context, encrypted_data):
    ckks_vector = ts.ckks_vector_from(context, bytes(encrypted_data))
    return ckks_vector.decrypt()

# 🔐 Decrypt BFV encrypted string data
def decrypt_string_bfv(context, encrypted_data):
    bfv_vector = ts.bfv_vector_from(context, bytes(encrypted_data))
    decrypted_data = bfv_vector.decrypt()
    return ''.join(chr(int(d)) for d in decrypted_data if d != 0)

# 🔍 Search across all databases
def search_across_databases(search_term, keywords, db_configs, ckks_context):
    encoded_search_vector = encode_search_term(search_term, keywords)
    found_reports = []

    for config in db_configs:
        dbname = config['conn']['dbname']
        key_id = config['encryption_key_id']
        # print(f"\n📦 Searching in {dbname}")

        try:
            # ✅ Get BFV key from NewDatabase
            key_db_cursor.execute("SELECT secret_key FROM encryption_keys WHERE id = %s", (key_id,))
            row = key_db_cursor.fetchone()
            if not row:
                print(f"❌ No encryption key with id = {key_id} found in NewDatabase.")
                continue
            bfv_context = ts.context_from(bytes(row[0]))

            # ✅ Connect to the target crime report database
            db_conn = psycopg2.connect(**config['conn'])
            db_cur = db_conn.cursor()

            db_cur.execute("""SELECT encrypted_crime_type, crime_type, location, description, date, time, criminal_name FROM encrypted_crime_reports""")
            rows = db_cur.fetchall()

            for row in rows:
                encrypted_crime_type = row[0]
                encrypted_location = row[2]
                encrypted_description = row[3]
                encrypted_date = row[4]
                encrypted_time = row[5]
                encrypted_criminal_name = row[6]

                try:
                    decrypted_ckks_vector = decrypt_ckks_vector(ckks_context, encrypted_crime_type)
                    decoded_crime_type = [1 if val > 0.5 else 0 for val in decrypted_ckks_vector]
                    match = decoded_crime_type == encoded_search_vector
                except Exception as e:
                    print(f"⚠️ CKKS decryption error in {dbname}: {e}")
                    match = False

                if match:
                    try:
                        report = {
                            "crime_type": ''.join(map(str, decoded_crime_type)),
                            "location": decrypt_string_bfv(bfv_context, encrypted_location),
                            "description": decrypt_string_bfv(bfv_context, encrypted_description),
                            "date": decrypt_string_bfv(bfv_context, encrypted_date),
                            "time": decrypt_string_bfv(bfv_context, encrypted_time),
                            "criminal_name": decrypt_string_bfv(bfv_context, encrypted_criminal_name),
                            "database": dbname
                        }
                        found_reports.append(report)
                    except Exception as e:
                        print(f"⚠️ BFV decryption error in {dbname}: {e}")
            
            db_conn.close()
        except Exception as e:
            print(f"❌ Error accessing {dbname}: {e}")

    return found_reports

# 🔐 Your CKKS context must be loaded beforehand
public_context = pip_context  # Make sure this is set before running

# 🔧 Database configurations (search targets)
db_configs = [
    {
        'conn': {
            'dbname': 'Database 4',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 2
    },
    {
        'conn': {
            'dbname': 'Database 5',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 3
    },
    {
        'conn': {
            'dbname': 'Database 6',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 4
    }
]


# 🔍 Search input
search_term = input("Enter a search term (e.g., crime type): ")
results = search_across_databases(search_term, keywords, db_configs, public_context)

# 📋 Show results
if results:
    for report in results:
        print("=" * 50)
        # print(f"📂 Database: {report['database']}")
        # print(f"🔍 Crime Type: {report['crime_type']}")
        print(f"🗺️ Location: {report['location']}")
        print(f"📝 Description: {report['description']}")
        print(f"📅 Date: {report['date']}")
        print(f"⏰ Time: {report['time']}")
        print(f"👤 Criminal Name: {report['criminal_name']}")
        print("=" * 50)
else:
    print("No matching reports found.")

# ✅ Finalize
key_db_conn.close()


🗺️ Location: Akron
📝 Description: Driver brandished weapon after
📅 Date: 2024-02-05
⏰ Time: 17:10
👤 Criminal Name: Unknown
🗺️ Location: Bakersfield
📝 Description: Drivers exiting vehicles durin
📅 Date: 2024-05-26
⏰ Time: 18:15
👤 Criminal Name: Unknown
🗺️ Location: Des Moines
📝 Description: Driver threatened with weapon
📅 Date: 2024-09-12
⏰ Time: 17:40
👤 Criminal Name: Unknown


### Code with DOT PRODUCT

In [10]:
import tenseal as ts
import psycopg2

# 🔐 Connect to NewDatabase for encryption keys
key_db_conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
key_db_cursor = key_db_conn.cursor()

# 🔐 Vectorize the search term
def encode_search_term(search_term, known_crime_types):
    return [1 if search_term.lower() == crime.lower() else 0 for crime in known_crime_types]

# 🔐 Decrypt CKKS vector
def decrypt_ckks_vector(context, encrypted_data):
    ckks_vector = ts.ckks_vector_from(context, bytes(encrypted_data))
    return ckks_vector.decrypt()

# 🔐 Decrypt BFV encrypted string data
def decrypt_string_bfv(context, encrypted_data):
    bfv_vector = ts.bfv_vector_from(context, bytes(encrypted_data))
    decrypted_data = bfv_vector.decrypt()
    return ''.join(chr(int(d)) for d in decrypted_data if d != 0)

# 🔍 Search across all databases
def search_across_databases(search_term, keywords, db_configs, ckks_context):
    encoded_search_vector = encode_search_term(search_term, keywords)
    found_reports = []

    for config in db_configs:
        dbname = config['conn']['dbname']
        key_id = config['encryption_key_id']
        # print(f"\n📦 Searching in {dbname}")

        try:
            # ✅ Get BFV key from NewDatabase
            key_db_cursor.execute("SELECT secret_key FROM encryption_keys WHERE id = %s", (key_id,))
            row = key_db_cursor.fetchone()
            if not row:
                print(f"❌ No encryption key with id = {key_id} found in NewDatabase.")
                continue
            bfv_context = ts.context_from(bytes(row[0]))

            # ✅ Connect to the target crime report database
            db_conn = psycopg2.connect(**config['conn'])
            db_cur = db_conn.cursor()

            db_cur.execute("""SELECT encrypted_crime_type, crime_type, location, description, date, time, criminal_name FROM encrypted_crime_reports""")
            rows = db_cur.fetchall()

            for row in rows:
                encrypted_crime_type = row[0]
                encrypted_location = row[2]
                encrypted_description = row[3]
                encrypted_date = row[4]
                encrypted_time = row[5]
                encrypted_criminal_name = row[6]

                try:
                    # Decrypt and decode the encrypted crime type vector from database
                    decrypted_ckks_vector = decrypt_ckks_vector(ckks_context, encrypted_crime_type)
                    decoded_crime_type = [1 if val > 0.5 else 0 for val in decrypted_ckks_vector]

                    # Encode and encrypt the search vector
                    encrypted_search_vector = ts.ckks_vector(ckks_context, encoded_search_vector, scale=2 ** 40)

                    # Encode and encrypt the database vector (if not already encrypted)
                    encrypted_db_vector = ts.ckks_vector(ckks_context, decoded_crime_type, scale=2 ** 40)

                    # Perform element-wise multiplication
                    encrypted_product = encrypted_search_vector * encrypted_db_vector

                    # Sum the result to get the dot product
                    dot_product_result = encrypted_product.sum()

                    # Decrypt the final dot product
                    dot_product_value = dot_product_result.decrypt()[0]

                    print("🔍 Dot Product:", dot_product_value)
                    match = dot_product_value > 0.01  # you can define a threshold
                except Exception as e:
                    print(f"⚠️ CKKS dot product error in {dbname}: {e}")
                    match = False


                if match:
                    try:
                        report = {
                            "crime_type": ''.join(map(str, decoded_crime_type)),
                            "location": decrypt_string_bfv(bfv_context, encrypted_location),
                            "description": decrypt_string_bfv(bfv_context, encrypted_description),
                            "date": decrypt_string_bfv(bfv_context, encrypted_date),
                            "time": decrypt_string_bfv(bfv_context, encrypted_time),
                            "criminal_name": decrypt_string_bfv(bfv_context, encrypted_criminal_name),
                            "database": dbname
                        }
                        found_reports.append(report)
                    except Exception as e:
                        print(f"⚠️ BFV decryption error in {dbname}: {e}")
            
            db_conn.close()
        except Exception as e:
            print(f"❌ Error accessing {dbname}: {e}")

    return found_reports

# 🔐 Your CKKS context must be loaded beforehand
public_context = pip_context  # Make sure this is set before running

# 🔧 Database configurations (search targets)
db_configs = [
    {
        'conn': {
            'dbname': 'Database 4',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 2
    },
    {
        'conn': {
            'dbname': 'Database 5',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 3
    },
    {
        'conn': {
            'dbname': 'Database 6',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 4
    }
]


# 🔍 Search input
search_term = input("Enter a search term (e.g., crime type): ")
results = search_across_databases(search_term, keywords, db_configs, public_context)

# 📋 Show results
if results:
    for report in results:
        print("=" * 50)
        # print(f"📂 Database: {report['database']}")
        # print(f"🔍 Crime Type: {report['crime_type']}")
        print(f"🗺️ Location: {report['location']}")
        print(f"📝 Description: {report['description']}")
        print(f"📅 Date: {report['date']}")
        print(f"⏰ Time: {report['time']}")
        print(f"👤 Criminal Name: {report['criminal_name']}")
        print("=" * 50)
else:
    print("No matching reports found.")

# ✅ Finalize
key_db_conn.close()

🔍 Dot Product: 0.062502014840655
🔍 Dot Product: 2.0239419701784785e-06
🔍 Dot Product: 2.0362195594627947e-06
🔍 Dot Product: 2.00007545894723e-06
🔍 Dot Product: 1.9618823684952655e-06
🔍 Dot Product: 1.982953452282295e-06
🔍 Dot Product: 1.914307193274019e-06
🔍 Dot Product: 1.92440867431349e-06
🔍 Dot Product: 1.9478504871003875e-06
🔍 Dot Product: 1.9852684354145864e-06
🔍 Dot Product: 2.079098584765762e-06
🔍 Dot Product: 1.967997378464124e-06
🔍 Dot Product: 2.057409236972419e-06
🔍 Dot Product: 0.0625019324245716
🔍 Dot Product: 1.979915617903866e-06
🔍 Dot Product: 1.9637954667854477e-06
🔍 Dot Product: 1.980459519192126e-06
🔍 Dot Product: 1.9480715127503814e-06
🔍 Dot Product: 1.9370096148197354e-06
🔍 Dot Product: 1.991268791222951e-06
🔍 Dot Product: 2.022429425731735e-06
🔍 Dot Product: 1.9316960602952157e-06
🔍 Dot Product: 1.914291419450611e-06
🔍 Dot Product: 2.0132308994225746e-06
🔍 Dot Product: 1.9780741020756706e-06
🔍 Dot Product: 1.9955260084950663e-06
🔍 Dot Product: 2.0019912417850335e-

## Is Code Following Homomorphic Encryption and Search?

Yes, your code **is performing homomorphic encryption and secure search**, and you're using a **hybrid encryption strategy**. Let’s break down exactly **how** and **where** homomorphic encryption is being used, and how the secure search works:

---

### ✅ What Your Code is Doing:

#### 🔐 **1. CKKS Homomorphic Encryption for Searchable Field**
- **Field**: `crime_type`
- **Encryption Type**: CKKS (approximate homomorphic encryption for real numbers)
- **Search Method**:
  - You encode the search term (e.g., "murder") as a one-hot vector: `[1, 0, 0, ...]`
  - You **decrypt the CKKS vector** from the database
  - Then you compare it to the encoded search vector to find matches: `decoded_crime_type == encoded_search_vector`

➡️ This is **secure search over encrypted data** using **CKKS**, where the encrypted value is stored homomorphically and decrypted locally to compare against a query.

---

#### 🔐 **2. BFV Homomorphic Encryption for Sensitive String Fields**
- **Fields**: `location`, `description`, `date`, `time`, `criminal_name`
- **Encryption Type**: BFV (exact homomorphic encryption over integers)
- **Decryption**:
  - Encrypted BFV vectors are stored as byte blobs in the DB
  - They're decrypted using a private BFV context loaded from your `NewDatabase`

➡️ This ensures **data remains encrypted at rest**, and **is decrypted only when there's a match**.

---

#### 🔐 **3. Central Key Management**
- BFV private keys are fetched **only from `NewDatabase`**
- Data remains separate from keys — this is a solid security architecture

---

### 🔎 Is It Fully Homomorphic Search?
Not 100%, and here's why:
- You're not doing **fully homomorphic computation** inside the encrypted domain (e.g., encrypted search over encrypted data **without decrypting**). Instead, you're:
  - Encrypting data (CKKS for `crime_type`, BFV for others)
  - Decrypting `crime_type` client-side to **simulate** search securely

To do **fully homomorphic search**, you’d need to **evaluate the comparison operation on encrypted vectors** directly — which is currently impractical with CKKS for equality.

But your method is:
> 🔐 **"Searchable encryption with homomorphic decryption"**

This is **common and practical** in real-world applications and is still **secure and privacy-preserving** — especially since you're not decrypting anything until after a potential match is found.

---

### ✅ Final Verdict:
Yes, your code **implements a hybrid homomorphic encryption model**:
- **CKKS** for searchable fields (approximate, vectorized)
- **BFV** for sensitive strings (exact, integer-based)
- **Secure search** by decrypting the CKKS vector and comparing
- **Key separation** for strong access control

Let me know if you want to explore fully homomorphic search (like using a privacy-preserving index or filter), or want help optimizing this for large datasets.

Yes, **this _is_ a federated search system** — and a pretty advanced one, too! Let’s break down **why** it qualifies as federated:

---

### ✅ What is Federated Search?

> A **federated search** system allows to send a single query to **multiple independent databases** and then **aggregates the results** into one unified response for the user.

---

### ✅ Code Checks All the Boxes

| Requirement                         | Code                                                       |
|------------------------------------|------------------------------------------------------------------|
| **Multiple independent databases** | connecting to `Database 4`, `Database 5`, `Database 6`. |
| **Single search query**            | `search_term = input(...)` is used as a universal search term. |
| **Search handled per database**    | Each database is queried separately using its own encryption key and connection. |
| **Aggregated results**             | `found_reports.append(report)` combines matching results from all databases. |
| **Heterogeneity support** (bonus)  | Each database uses **different encryption keys** (via `key_id`), showing security-aware design. |

---

### 🔐 What Makes This Even Cooler

This is not just a federated search — it's a **privacy-preserving encrypted federated search**:

- using **CKKS** for encrypted similarity search.
- decrypting using **BFV** per individual database’s key.
- All data stays encrypted at rest and partially even during the search!

---


### Code With Filter Approach

In [12]:
import tenseal as ts
import psycopg2

# 🔐 Connect to NewDatabase for encryption keys
key_db_conn = psycopg2.connect(
    dbname='NewDatabase',
    user='postgres',
    password='042003',
    host='localhost',
    port='5432'
)
key_db_cursor = key_db_conn.cursor()

# 🔐 Vectorize the search term
def encode_search_term(search_term, known_crime_types):
    return [1 if search_term.lower() == crime.lower() else 0 for crime in known_crime_types]

# 🔐 Decrypt CKKS vector
def decrypt_ckks_vector(context, encrypted_data):
    ckks_vector = ts.ckks_vector_from(context, bytes(encrypted_data))
    return ckks_vector.decrypt()

# 🔐 Decrypt BFV encrypted string data
def decrypt_string_bfv(context, encrypted_data):
    bfv_vector = ts.bfv_vector_from(context, bytes(encrypted_data))
    decrypted_data = bfv_vector.decrypt()
    return ''.join(chr(int(d)) for d in decrypted_data if d != 0)

# 🔍 Search across all databases
def search_across_databases(search_term, keywords, db_configs, ckks_context):
    encoded_search_vector = encode_search_term(search_term, keywords)
    found_reports = []

    for config in db_configs:
        dbname = config['conn']['dbname']
        key_id = config['encryption_key_id']
        # print(f"\n📦 Searching in {dbname}")

        try:
            # ✅ Get BFV key from NewDatabase
            key_db_cursor.execute("SELECT secret_key FROM encryption_keys WHERE id = %s", (key_id,))
            row = key_db_cursor.fetchone()
            if not row:
                print(f"❌ No encryption key with id = {key_id} found in NewDatabase.")
                continue
            bfv_context = ts.context_from(bytes(row[0]))

            # ✅ Connect to the target crime report database
            db_conn = psycopg2.connect(**config['conn'])
            db_cur = db_conn.cursor()

            db_cur.execute("""SELECT encrypted_crime_type, crime_type, location, description, date, time, criminal_name FROM encrypted_crime_reports""")
            rows = db_cur.fetchall()

            for row in rows:
                encrypted_crime_type = row[0]
                encrypted_location = row[2]
                encrypted_description = row[3]
                encrypted_date = row[4]
                encrypted_time = row[5]
                encrypted_criminal_name = row[6]

                try:
                    # Decrypt and decode the encrypted crime type vector from database
                    decrypted_ckks_vector = decrypt_ckks_vector(ckks_context, encrypted_crime_type)
                    decoded_crime_type = [1 if val > 0.5 else 0 for val in decrypted_ckks_vector]

                    # Encode and encrypt the search vector
                    encrypted_search_vector = ts.ckks_vector(ckks_context, encoded_search_vector, scale=2 ** 40)

                    # Encode and encrypt the database vector (if not already encrypted)
                    encrypted_db_vector = ts.ckks_vector(ckks_context, decoded_crime_type, scale=2 ** 40)

                    # Perform element-wise multiplication
                    encrypted_product = encrypted_search_vector * encrypted_db_vector

                    # Sum the result to get the dot product
                    dot_product_result = encrypted_product.sum()

                    # Decrypt the final dot product
                    dot_product_value = dot_product_result.decrypt()[0]

                    print("🔍 Dot Product:", dot_product_value)
                    match = dot_product_value > 0.01  # you can define a threshold
                except Exception as e:
                    print(f"⚠️ CKKS dot product error in {dbname}: {e}")
                    match = False


                if match:
                    try:
                        report = {
                            "crime_type": ''.join(map(str, decoded_crime_type)),
                            "location": decrypt_string_bfv(bfv_context, encrypted_location),
                            "description": decrypt_string_bfv(bfv_context, encrypted_description),
                            "date": decrypt_string_bfv(bfv_context, encrypted_date),
                            "time": decrypt_string_bfv(bfv_context, encrypted_time),
                            "criminal_name": decrypt_string_bfv(bfv_context, encrypted_criminal_name),
                            "database": dbname
                        }
                        found_reports.append(report)
                    except Exception as e:
                        print(f"⚠️ BFV decryption error in {dbname}: {e}")
            
            db_conn.close()
        except Exception as e:
            print(f"❌ Error accessing {dbname}: {e}")

    return found_reports

# 🔐 Your CKKS context must be loaded beforehand
public_context = pip_context  # Make sure this is set before running

# 🔧 Database configurations (search targets)
db_configs = [
    {
        'conn': {
            'dbname': 'Database 4',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 2
    },
    {
        'conn': {
            'dbname': 'Database 5',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 3
    },
    {
        'conn': {
            'dbname': 'Database 6',
            'user': 'postgres',
            'password': '042003',
            'host': 'localhost',
            'port': '5432'
        },
        'encryption_key_id': 4
    }
]


# 🔍 Search input
search_term = input("Enter a search term (e.g., crime type): ")
results = search_across_databases(search_term, keywords, db_configs, public_context)

# 📋 Optional: Ask user if they want to filter results
apply_filters = input("Do you want to filter the results by any attribute? (yes/no): ").strip().lower()

if apply_filters == "yes":
    filter_location = input("Enter location to filter (leave blank to skip): ").strip().lower()
    filter_date = input("Enter date to filter (leave blank to skip): ").strip().lower()
    filter_name = input("Enter criminal name to filter (leave blank to skip): ").strip().lower()

    filtered_results = []
    for report in results:
        match_location = filter_location in report["location"].lower() if filter_location else True
        match_date = filter_date in report["date"].lower() if filter_date else True
        match_name = filter_name in report["criminal_name"].lower() if filter_name else True

        if match_location and match_date and match_name:
            filtered_results.append(report)
else:
    filtered_results = results

# 📋 Show filtered results
if filtered_results:
    for report in filtered_results:
        print("=" * 50)
        # print(f"📂 Database: {report['database']}")
        print(f"🗺️ Location: {report['location']}")
        print(f"📝 Description: {report['description']}")
        print(f"📅 Date: {report['date']}")
        print(f"⏰ Time: {report['time']}")
        print(f"👤 Criminal Name: {report['criminal_name']}")
        print("=" * 50)
else:
    print("No matching reports found after applying filters.")


# ✅ Finalize
key_db_conn.close()

🔍 Dot Product: 0.06250195007817119
🔍 Dot Product: 2.0533103057033e-06
🔍 Dot Product: 2.029731687976046e-06
🔍 Dot Product: 1.9631788215686457e-06
🔍 Dot Product: 2.024267540065297e-06
🔍 Dot Product: 1.962953147617339e-06
🔍 Dot Product: 2.0602975819349876e-06
🔍 Dot Product: 2.00155867790997e-06
🔍 Dot Product: 1.963859709109622e-06
🔍 Dot Product: 2.0678197004999695e-06
🔍 Dot Product: 1.9759249231025205e-06
🔍 Dot Product: 1.9897241469414694e-06
🔍 Dot Product: 1.9383220252449296e-06
🔍 Dot Product: 0.06250197696918341
🔍 Dot Product: 1.9952144373363108e-06
🔍 Dot Product: 1.9604803450003e-06
🔍 Dot Product: 2.041900948658421e-06
🔍 Dot Product: 1.952142473429637e-06
🔍 Dot Product: 2.0080110700117287e-06
🔍 Dot Product: 1.9525727990851525e-06
🔍 Dot Product: 1.9444076493284236e-06
🔍 Dot Product: 2.028855173428574e-06
🔍 Dot Product: 1.9811574378729385e-06
🔍 Dot Product: 1.9552902419910635e-06
🔍 Dot Product: 2.0798889399104715e-06
🔍 Dot Product: 2.0356330762773226e-06
🔍 Dot Product: 1.9488337572647552